<a href="https://colab.research.google.com/github/cl16908/models_on_pnr1z/blob/main/GPD_run_pnr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Runs the GPD model (Ross et al, 2018) on PNR-1z data

#### Cindy Lim Shin Yee
###### Last updated: 21st July 2021
---

The GPD model in this notebook is developed by Ross et al (2018).
More information and tutorials about the GPD model can be found here: https://github.com/interseismic/generalized-phase-detection

Input:
The seismic dataset provided is from the PNR-1z dataset.

Output:
Event catalogue with event origin times, longitude, latitude, depth, Easting and Northing coordinates.

References:

- Ross, Z.E., Meier, M.A., Hauksson, E. and Heaton, T.H., 2018. <i> Generalized seismic phase detection with deep learning. </i> Bulletin of the Seismological Society of America, 108(5A), pp.2894-2901. https://doi.org/10.1785/0120180080

## Installing obspy

In [ ]:
!pip install obspy

In [ ]:
!pip install pyproj

In [ ]:
!pip install h5py==2.10.0

# Importing modules

In [ ]:
# Original GPD model was produced using TensorFlow (TF) version 1, so we need to tell Google Colab to use v1 (comment out this step if using conda environment above)
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import string
import time
import argparse as ap
import sys
import os

import numpy as np
import obspy.core as oc
from obspy.signal.trigger import trigger_onset
from obspy.core import read
import obspy.signal
import math


import matplotlib as mpl
import pylab as plt
import pandas as pd
from obspy import UTCDateTime

import keras
from keras.models import *
from keras.layers import *
from keras import optimizers
from keras import losses
from keras.models import *
import tensorflow as tf

import glob
import h5py
import random
import scipy
import pandas as pd
import datetime as datetime
import pyproj


In [ ]:
# For interpolating along well path (needed for locating and orienting stations)
from scipy.interpolate import InterpolatedUnivariateSpline

from tensorflow.python.client import device_lib

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# Define functions

In [ ]:
# Define function to produce sliding windows of signal (from original GPD GitHub repo: https://github.com/interseismic/generalized-phase-detection/blob/master/gpd_predict.py)

def sliding_window(data, size, stepsize=1, padded=False, axis=-1, copy=True):
    """
    Calculate a sliding window over a signal
    Parameters
    ----------
    data : numpy array
        The array to be slided over.
    size : int
        The sliding window size
    stepsize : int
        The sliding window stepsize. Defaults to 1.
    axis : int
        The axis to slide over. Defaults to the last axis.
    copy : bool
        Return strided array as copy to avoid sideffects when manipulating the
        output array.
    Returns
    -------
    data : numpy array
        A matrix where row in last dimension consists of one instance
        of the sliding window.
    Notes
    -----
    - Be wary of setting `copy` to `False` as undesired sideffects with the
      output values may occurr.
    Examples
    --------
    >>> a = numpy.array([1, 2, 3, 4, 5])
    >>> sliding_window(a, size=3)
    array([[1, 2, 3],
           [2, 3, 4],
           [3, 4, 5]])
    >>> sliding_window(a, size=3, stepsize=2)
    array([[1, 2, 3],
           [3, 4, 5]])
    See Also
    --------
    pieces : Calculate number of pieces available by sliding
    """
    if axis >= data.ndim:
        raise ValueError(
            "Axis value out of range"
        )

    if stepsize < 1:
        raise ValueError(
            "Stepsize may not be zero or negative"
        )

    if size > data.shape[axis]:
        raise ValueError(
            "Sliding window size may not exceed size of selected axis"
        )

    shape = list(data.shape)
    shape[axis] = np.floor(data.shape[axis] / stepsize - size / stepsize + 1).astype(int)
    shape.append(size)

    strides = list(data.strides)
    strides[axis] *= stepsize
    strides.append(data.strides[axis])

    strided = np.lib.stride_tricks.as_strided(
        data, shape=shape, strides=strides
    )

    if copy:
        return strided.copy()
    else:
        return strided

In [ ]:
# Defining file reading function (from Alan Baird, October 2019)
def readPNR(fname,**kwargs):
    """Wrapper function for reading PNR segy data which sets headers for
    stations and channels automatically"""
    st = read(fname,**kwargs)
    # populate trace headers for network, station name, channel, and distance (depth, needed to easily produce section plots)
    for i, trace in enumerate(st):
        trace.stats.network = "PNR"
        trace.stats.station = "{:0>3d}".format(trace.stats.segy.trace_header.geophone_group_number_of_last_trace)
        trace.stats.channel = 'BS'+str(i%3+1)
        trace.stats.distance = trace.stats.segy.trace_header.receiver_group_elevation/1000
    return st

def readPNR_ENZ(fname,**kwargs):
    """Wrapper function for reading PNR segy data and reorienting it into ENZ
    components witgh appropriate headers.

    Note: Requires that the stns dataframe has been read in correctly"""
    st = readPNR(fname,**kwargs)

    # Create a copy of the traces to be rotated
    strot=st.copy()

    # Loop through stations, construct rotation matrices and apply to apply to traces
    for i, row in stns.iterrows():
        azi=np.radians(row['azi'])
        inc=np.radians(row['inc'])
        rot=np.radians(row['rgb_orient'])

        # Create rotation matrices
        R1=np.array([[np.sin(azi), -np.cos(azi), 0],
                     [np.cos(azi),  np.sin(azi), 0],
                     [          0,            0, 1]])
        R2=np.array([[np.cos(inc), 0, -np.sin(inc)],
                     [          0, 1,            0],
                     [np.sin(inc), 0,  np.cos(inc)]])
        R3=np.array([[ np.cos(rot), np.sin(rot), 0],
                     [-np.sin(rot), np.cos(rot), 0],
                     [           0,           0, 1]])

        # Select traces from appropriate station
        stnm = "{:0>3d}".format(int(row['name']))
        sttmp = strot.select(station=stnm)

        # instrument coordinates are left-handed Hx,Hy,Vz, need to flip polarization of Hy to make it right handed
        tmparrays = np.stack((sttmp[0].data,-sttmp[1].data,sttmp[2].data))

        # Apply the rotation
        rotated = R1 @ R2 @ R3 @ tmparrays

        # Set the data of the traces
        sttmp[0].data=rotated[0,:]
        sttmp[1].data=rotated[1,:]
        sttmp[2].data=rotated[2,:]

        # Rename the channels
        sttmp[0].stats.channel = 'BSE'
        sttmp[1].stats.channel = 'BSN'
        sttmp[2].stats.channel = 'BSZ'
    return strot

## Hyper-parameters

In [ ]:
# Hyperparameters
# INPUT
freq_max = 50.0
filter_data = True
#decimate_data = False # If false, assumes data is already 100 Hz sample rate
n_shift = 50 # Number of samples to shift the sliding window at a time
n_gpu = 1 # Number of GPUs to use (if any)
batch_size = 100*3

half_dur = 0.1
only_dt = 1/2000
n_win = int(half_dur/only_dt)
n_feat = 2*n_win

# Model prediction parameters
shift_size = 50 # sliding window shift size
batch_size = 100 # how many samples are read into the model at a time

##########################################################################################

# Trigger parameters - I've set a lowish value but can then just look at ones with higher thresholds later if I prefer
p_thresh = 0.3 # 0.5 means that P should have the highest probability (S and noise combined will be less than 0.5)
s_thresh = 0.3 # Ditto for S
trig_off = 0.2 # Turn off trigger if probability drops below 0.2 - this may need to be higher if the background probs for P and S are generally hovering around this level

if n_gpu > 1:
   from keras.utils import multi_gpu_model
   model = multi_gpu_model(model, gpus=n_gpu)
   print("multiple GPU's loaded")

def get_available_devices():
   local_device_protos = device_lib.list_local_devices()
   return [x.name for x in local_device_protos]
print(get_available_devices())

['/device:CPU:0', '/device:XLA_CPU:0', '/device:XLA_GPU:0', '/device:GPU:0']


## Download and load trained GPD model

In [ ]:
!git clone https://github.com/interseismic/generalized-phase-detection.git

fatal: destination path 'generalized-phase-detection' already exists and is not an empty directory.


In [ ]:
# load model json and create model
json_file = open("./generalized-phase-detection/model_pol.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json, custom_objects={'tf':tf})
print("****GPD MODEL LOADED FROM DISK****")

# load weights into new model
model.load_weights( "./generalized-phase-detection/model_pol_best.hdf5")
print("****ORIGINAL MODEL AND WEIGHTS LOADED FROM DISK*****")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

****GPD MODEL LOADED FROM DISK****
****ORIGINAL MODEL AND WEIGHTS LOADED FROM DISK*****


## Get the ground-truth (CMM) catalogue and obtain the continuous data

In [ ]:
!git clone https://github.com/cl16908/PNR_run.git

# Define some paths
drive = './PNR_run/'

# Read the station orientations for trace rotations
# read in event catalogue as a dataframe (note that these don't have traveltime picks)
df_cat = pd.read_csv(drive + '/Catalog/Event_and_Station/PNR-1z_FullCatalogue.dat',
                    delim_whitespace=True)

# create a new datetime column with the origin times of the events
df_cat['datetime']=df_cat['T'].apply(UTCDateTime) + df_cat['QC_LOC_T0']

# well path 1
wp1 = pd.read_csv(drive + '/Catalog/Event_and_Station/PNR-1z_Wellpath.dat', delim_whitespace=True)

# well path 2
wp2 = pd.read_csv(drive + '/Catalog/Event_and_Station/PNR-2_Wellpath.dat', delim_whitespace=True)

# location of the injection sleeves of pnr1 and 2
pnr1_stgs = pd.read_csv(drive + '/Catalog/Event_and_Station/PNR-1z_Stages.dat', delim_whitespace=True)
pnr2_stgs = pd.read_csv(drive + '/Catalog/Event_and_Station/PNR-2_Stages.dat', delim_whitespace=True)

# station locations and "orientations"
stns = pd.read_csv(drive + '/Catalog/Event_and_Station/PNR-1z_Stations_orient.dat', delim_whitespace=True)


# Functions to return well inclination and azimuth from measured depth
md2inc=InterpolatedUnivariateSpline(wp2['MD'],wp2['Inc'])
md2az=InterpolatedUnivariateSpline(wp2['MD'],wp2['Az'])


# Find measured depth for each station by root finding
MDs=[]
for stdp in stns['stz']:
    MDs.append(InterpolatedUnivariateSpline(wp2['MD'],wp2['TVD_MSL']-stdp).roots()[0])

# assign station measured depth and compute station azimuth and inclination
stns['MD'] = MDs
stns['azi']=stns['MD'].apply(md2az)
stns['inc']=stns['MD'].apply(md2inc)

fatal: destination path 'PNR_run' already exists and is not an empty directory.


In [ ]:
# Reading filenames of the segy files

this_folder = '181211'
hour = '09'
filenames1 = int('20'+this_folder + hour)
filenames = drive + '/data/' + this_folder[0:6] + '/' + str(filenames1) + '*.segy'
fhour = glob.glob(filenames)
fhour.sort()

In [ ]:
# PRE-ALLOCATE instead
ahour = np.zeros((24,2000*16*len(fhour),3))

try:
  for j in np.arange(0,len(fhour)):
      sthour = readPNR_ENZ(fhour[j],unpack_trace_headers = True)
      for i in np.arange(0,len(sthour)):
          ahour[math.floor(i/3),np.arange(j*2000*16,(j+1)*2000*16),(i%3)] = sthour[i].data
except:
  print("***Please restart notebook from beginning***")

# TIME MATRIX TOTAL
t = np.arange(0,len(fhour)*16,0.0005) # from 0 to 3600 seconds, 0.0005 interval
st = sthour
a = ahour
dt = 0.0005

stations = list(set([st[s].stats.station for s in np.arange(0, len(st))]))
stations.sort()

# INPUT FIRST FILE NUMBER NAME
name = fhour[0]

# set starttime
for i in np.arange(0,len(st)):
    st[i].stats.starttime = UTCDateTime(int(name[-23:-19]), int(name[-19:-17]), int(name[-17:-15]), int(name[-15:-13]), int(name[-13:-11]), int(name[-11:-9]))

for s in np.arange(0,len(stations)):
    st2 = st.select(station=stations[s])

chan = st2[0].stats.channel
sr = st2[0].stats.sampling_rate
dt = st2[0].stats.delta
net = st2[0].stats.network
sta = st2[0].stats.station

for i in np.arange(0,(ahour.shape[0])-1):
    a[i,:,0] = obspy.signal.detrend.simple(a[i,:,0])
    a[i,:,1] = obspy.signal.detrend.simple(a[i,:,1])
    a[i,:,2] = obspy.signal.detrend.simple(a[i,:,2])
    

In [ ]:
# filter data matrix
if filter_data:
    import obspy.signal.filter as obs
    for i in np.arange(0,(ahour.shape[0])-1):
        a[i,:,0] = obs.highpass(a[i,:,0],freq=freq_max,df=st[0].stats.sampling_rate)
        a[i,:,1] = obs.highpass(a[i,:,1], freq=freq_max, df=st[0].stats.sampling_rate)
        a[i,:,2] = obs.highpass(a[i,:,2], freq=freq_max, df=st[0].stats.sampling_rate)

## Running the GPD model on the catalogue



In [ ]:
## MODEL RUNNING - takes about x minutes for 1 PNR hour (~2GB)
df = pd.DataFrame(columns=['time', 'sta', 'pha','prob']) # To store picks

# Loop through the list of stations
before = datetime.datetime.now() # Get time before and after to see how long this takes
for s in np.arange(0, a.shape[0]):

    p_picks = []
    p_probs = []
    s_picks = []
    s_probs = []

    # Set up window values (by the look of it)
    tt = (np.arange(0, a.shape[1], n_shift) + n_win) * dt # centre of window (in secs)
    tt_i = np.arange(0, a.shape[1], n_shift) + n_feat # end of window? (in samples)

    # Get sliding windows for each component of first station
    # sliding_N = sliding_window(a.shape[s,:,1], n_feat, stepsize=n_shift)
    
    sliding_E = sliding_window(a[s,:,0], n_feat, stepsize=n_shift)
    sliding_N = sliding_window(a[s,:,1], n_feat, stepsize=n_shift)
    sliding_Z = sliding_window(a[s,:,2], n_feat, stepsize=n_shift)

    # Then put in 3D array (no. of windows x length of window x 3 channels) for model input
    # Works better if broken up (and free up memory by removing sliding_* variables...)
    tr_win = np.zeros((sliding_N.shape[0], n_feat, 3))
    tr_win[:,:,1] = sliding_N
    sliding_N = None # clear some memory
    tr_win[:,:,0] = sliding_E
    sliding_E = None # clear some memory
    tr_win[:,:,2] = sliding_Z
    sliding_Z = None

    # Normalize between 0 and 1
    tr_win = tr_win / np.max(np.abs(tr_win), axis=(1,2))[:,None,None]

    # Not 100% sure what this and next step do...
    tt = tt[:tr_win.shape[0]]
    tt_i = tt_i[:tr_win.shape[0]]

    # Model prediction step... here goes nothing!
    ts = model.predict(tr_win, verbose=True, batch_size=batch_size)

    # Interpolate predictions so same sample rate as signal:
    ts_times = np.arange(0, a.shape[1])
    # class_trace = np.zeros((a.shape[1], int(3))) # Create prediction traces to be same size as input data
    # for i in np.arange(0, 3): # For each class
    #     class_trace[:, i] = np.interp(ts_times, np.arange(199, a.shape[1]-200, n_shift), ts[:,i]) # Interpolate prediction values (ts)
    trigs = trigger_onset(ts[:,0],p_thresh,trig_off)

    for trig in trigs: # For each trigger
        if trig[1] == trig[0]: # if the trigger on an off times are the same then go on to the next one
            continue
        pick = np.argmax(ts[trig[0]:trig[1],0]) + trig[0] # the pick is the sample with max probability between the trigger on and off times
        stamp_pick = st[0].stats.starttime + (((pick * shift_size) + 200) * st[0].stats.delta) # this should give the correct UTc time of the pick
        p_picks.append(stamp_pick) # add the pick time to p_picks list
        p_probs.append(ts[pick,0]) # add the pick probability to p_probs list
    # S-wave picks (trigger)

    trigs = trigger_onset(ts[:,1],s_thresh,trig_off)
    for trig in trigs:
        if trig[1] == trig [0]:
            continue
        pick = np.argmax(ts[trig[0]:trig[1],1]) + trig[0]
        stamp_pick = st[0].stats.starttime + (((pick * shift_size) + 200) * st[0].stats.delta)
        s_picks.append(stamp_pick)
        s_probs.append(ts[pick,1])

    if len(p_picks) > 0: # If you have P-wave picks
        df_add = pd.DataFrame(p_picks, columns=['time']) # Create a new dataframe with pick times
        df_add['sta'] = stations[s] # Add the station name for these picks
        df_add['pha'] = 'p' # Add the phase for these picks
        df_add['prob'] = p_probs # Add the probabilities for these picks
        df = pd.concat([df, df_add]) # Join it to the main pick dataframe (all stations)

    if len(s_picks) > 0: # Same as above for S-wave picks
        df_add = pd.DataFrame(s_picks, columns=['time'])
        df_add['sta'] = stations[s]
        df_add['pha'] = 's'
        df_add['prob'] = s_probs
        df = pd.concat([df, df_add])
 

df = df.sort_values(by='time').reset_index(drop=True) # After running the above for all stations, sort the pick dataframe by UTC time
# df.to_csv(root + 'output_folder_multi/' + test_folder + '/output/' + 'OTS.' + this_folder + '.' +str(filenames1)  + "_trigs.csv")

after = datetime.datetime.now()
time_taken = after-before
print("1 hour data from " + str(a.shape[0]) + " stations processed in " + str(time_taken.seconds) + "." + str(time_taken.microseconds) + " seconds.")    



143993/143993 [==============================] - 32s 219us/step
1 hour data from 24 stations processed in 728.528165 seconds.


In [ ]:
df

,time,sta,pha,prob
0,2018-12-11T09:00:35.975000Z,015,s,0.459584
1,2018-12-11T09:00:37.200000Z,011,p,0.546031
2,2018-12-11T09:00:55.075000Z,020,p,0.392744
3,2018-12-11T09:00:55.925000Z,022,p,0.541089
4,2018-12-11T09:00:58.150000Z,013,p,0.913878
...,...,...,...,...
41199,2018-12-11T10:00:00.000000Z,007,s,0.915917
41200,2018-12-11T10:00:00.325000Z,013,s,0.409411
41201,2018-12-11T10:00:01.700000Z,023,s,0.858782
41202,2018-12-11T10:00:01.950000Z,005,s,0.322514


## Phase Association for event location

In [ ]:
###############################
# Grouping Ps
###############################
print('Grouping Ps.')
# Dataframe with only Ps
df_P = df[df['pha']=='p']
df_P = df_P.reset_index(drop=True)

# convert all string 'time' entries to UTCDateTime

for i in np.arange(0,len(df_P)):
    df_P['time'].values[i] = UTCDateTime(df_P['time'].values[i])

timewin = 0.2 # time window for 'event' groups
count = 0
ngroup = 1 # set group number to 0
group = pd.DataFrame(columns=['time','sta','pha','prob']) # set an empty 'group' dataframe
j = 0 # will be the 'header' event time (earliest time in the event group for naming)
Pevs = pd.DataFrame(columns=['time','sta','pha','prob','group_no'])

# loop through the whole df_P (2 mins 30s/ 1MIN 10S)
for i in np.arange(0,len(df_P)):
    # if the timestamp is <= the upper time limit, add details to 'group_add' df to be concatenated to 'group' df   
    if (df_P['time'].values[i] <= df_P['time'].values[j]+timewin):
        count += 1
        group_add = pd.DataFrame(index= np.arange(0,1),columns=['time','sta','pha','prob'])
        group_add['time'] = df_P['time'].values[i] 
        group_add['sta'] = df_P['sta'].values[i]
        group_add['pha'] = df_P['pha'].values[i]
        group_add['prob'] = df_P['prob'].values[i]
        group = pd.concat([group,group_add])   

    # if time is > the upper time limit, it is a new event group
    else:
      # if the group length is >= 4, add to PSevs
        if len(group)>= 4:
          group['group_no'] = [ngroup] * len(group)
          Pevs = pd.concat([Pevs, group])
          ngroup += 1
        else:
          pass
      
        # j index will be the first/header event time name, increase by i for the next header event time
        j = i

        # make new group, with the new header event as first entry 
        group = pd.DataFrame(columns=['time','sta','pha','prob']) # empty df
        group_add['time'] = df_P['time'].values[i] 
        group_add['sta'] = df_P['sta'].values[i]
        group_add['pha'] = df_P['pha'].values[i]
        group_add['prob'] = df_P['prob'].values[i]
        group = pd.concat([group,group_add])

# out of loop, save the last group
j = i
if len(group)>= 4:
  group['group_no'] = [ngroup] * len(group)
  Pevs = pd.concat([Pevs, group])
  ngroup += 1

Grouping Ps.


In [ ]:
###############################
# Grouping Ss - takes about 2 minutes
###############################
print('Grouping Ss.')
df_S = df[df['pha']=='s']
df_S = df_S.reset_index(drop=True)

# converting all string entries into UTCDateTime
for i in np.arange(0,len(df_S)):
    df_S['time'].values[i] = UTCDateTime(df_S['time'].values[i])

timewin = 0.2
count = 0
ngroup = 1
group = pd.DataFrame(columns=['time','sta','pha','prob'])
j = 0
Sevs = pd.DataFrame(columns=['time','sta','pha','prob','group_no'])

# loop through the whole df_S (2 mins 30s/ 1MIN45S)
for i in np.arange(0,len(df_S)):
    # if the timestamp is <= the upper time limit, add details to 'group_add' df to be concatenated to 'group' df   
    if (df_S['time'].values[i] <= df_S['time'].values[j]+timewin):
        count += 1
        group_add = pd.DataFrame(index= np.arange(0,1),columns=['time','sta','pha','prob'])
        group_add['time'] = df_S['time'].values[i] 
        group_add['sta'] = df_S['sta'].values[i]
        group_add['pha'] = df_S['pha'].values[i]
        group_add['prob'] = df_S['prob'].values[i]
        group = pd.concat([group,group_add])   

    # if time is > the upper time limit, it is a new event group
    else:
      # if the group length is >= 4, add to PSevs
        if len(group)>= 4:
          group['group_no'] = [ngroup] * len(group)
          Sevs = pd.concat([Sevs, group])
          ngroup += 1
        else:
          pass
      
        # j index will be the first/header event time name, increase by i for the next header event time
        j = i

        # make new group, with the new header event as first entry 
        group = pd.DataFrame(columns=['time','sta','pha','prob']) # empty df
        group_add['time'] = df_S['time'].values[i] 
        group_add['sta'] = df_S['sta'].values[i]
        group_add['pha'] = df_S['pha'].values[i]
        group_add['prob'] = df_S['prob'].values[i]
        group = pd.concat([group,group_add])

# out of loop, save the last group
j = i
if len(group)>= 4:
  group['group_no'] = [ngroup] * len(group)
  Sevs = pd.concat([Sevs, group])
  ngroup += 1

Grouping Ss.


In [ ]:
# associating S to the Ps
# if Stime between Ptime and Ptime + 0.13
# if  Ptime <= Stime <= Ptime + 0.13
print('Phase associating.')
PSevs = Pevs.copy()
PSevs['stime'] = np.nan * len(PSevs)
Stime = []
# add Pevs['prob_s']
PSevs['prob_s'] = np.nan * len(PSevs)
# PSevs reset index
PSevs = PSevs.reset_index(drop=True)

# for all P phase entries, search through the S arrival picks
for j in np.arange(0,len(PSevs)):
    Stime = []
    # same station in Sevs dataframe
    samesta_df = Sevs[Sevs['sta']==PSevs['sta'].iloc[j]]
    samesta_df = samesta_df.reset_index(drop=True)

    for i in np.arange(0,len(samesta_df)):
        # if within time window, add to Stime matrix
      if (samesta_df['time'].iloc[i] <= PSevs['time'].iloc[j] + 0.13) and (samesta_df['time'].iloc[i] >= PSevs['time'].iloc[j]):
        Stime.append(samesta_df['time'].iloc[i])
            
        # if S picks have multiple times, take the higher prob_S
        if len(Stime)>1:
                
          # if new Sevs prob > prob already set, overwrite:
          if samesta_df['prob'].iloc[i] > PSevs['prob_s'].iloc[j]:
              PSevs['stime'].iloc[j] = samesta_df['time'].iloc[i]
              PSevs['prob_s'].iloc[j] = samesta_df['prob'].iloc[i]
              # print('added! x2')
              
          else:
              pass
        # else add to PSevs dataframe
        else:
            
            PSevs['stime'].iloc[j] = samesta_df['time'].iloc[i]
            PSevs['prob_s'].iloc[j] = samesta_df['prob'].iloc[i]

         
    else:
        pass          

PSevs['pha_s'] = ['s'] * len(PSevs)

# resetting index
PSevs = PSevs.reset_index(drop=True)

Phase associating.


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
# add a null space after every Stime
idxnan = []
for i in np.arange(0,len(PSevs)):
    if pd.isnull(PSevs['stime'].iloc[i]):
        pass
    else:
        idxnan.append(i)

for i in np.arange(0,len(idxnan)):
    idxnan[i] = idxnan[i] + .5

idxnew = PSevs.index.union(idxnan)[:-1]
PSevs = PSevs.reindex(idxnew).reset_index(drop=True)

# if time value is nan, fill in with details from before
for i in np.arange(1,len(PSevs)):
    
    if pd.isnull(PSevs['time'].iloc[i]):
        PSevs['time'].values[i] = PSevs['stime'].values[i-1]
        PSevs['sta'].values[i] = PSevs['sta'].values[i-1]
        PSevs['pha'].values[i] = PSevs['pha_s'].values[i-1]
        PSevs['prob'].values[i] = PSevs['prob_s'].values[i-1]
        PSevs['group_no'].values[i] = PSevs['group_no'].values[i-1]
    else:
        pass

# empty space after every group no
idx = PSevs.index.union(PSevs.index[PSevs['group_no'].shift(-1).ne(PSevs['group_no'])] + .5)[:-1]

PSevs = PSevs.reindex(idx).reset_index(drop=True)

# dropping all unnecessary columns
PSevs = PSevs.drop(columns=['stime','prob_s','pha_s','group_no'])

cols = ['sta','pha','prob','time']

PSevs = PSevs[cols]

In [ ]:
PSevs.iloc[0:20]

,sta,pha,prob,time
0,PR17IP,p,0.663571,181211090105.92500
1,PR17IP,s,0.906770,181211090105.97500
2,PR13IP,p,0.471594,181211090105.95000
3,PR12IP,p,0.416472,181211090105.97500
4,PR03IP,p,0.470835,181211090106.12500
5,NaN,NaN,NaN,n
6,PR24IP,p,0.770744,181211091505.27500
7,PR24IP,s,0.968348,181211091505.32500
8,PR13IP,p,0.621063,181211091505.32500
9,PR13IP,s,0.995397,181211091505.40000


In [ ]:
##################################
# OBSERVATION FILE MAKER
##################################

# new dataframe for nlloc
PSobs = pd.DataFrame(columns=['sta','instrument','comp','Ponset','phase','firstmotion','date(yyymmdd)','hhmm','ss','err','errmag','coda_duration','amplitude','period','priorwt'])

# convert all UTCDateTime times to string
for i in np.arange(0,len(PSevs)):

  PSevs['time'].values[i] = str(PSevs['time'].values[i])

  if isinstance(PSevs['time'].values[i],str):
      PSevs['time'].values[i] = PSevs['time'].values[i][2:4] + PSevs['time'].values[i][5:7] + PSevs['time'].values[i][8:10] + PSevs['time'].values[i][11:13] + PSevs['time'].values[i][14:16] + PSevs['time'].values[i][17:25]
      # PSevs['time'].values[i] = '{:07.4f}'.format(PSevs['time'].values[i])
  else:
      pass

# making sure each station number is at least 2 characters

for i in np.arange(0,len(PSevs)):
    if (not isinstance(PSevs['sta'].iloc[i],str)) and (PSevs['sta'].iloc[i] < 10):
        PSevs['sta'].iloc[i] = '0' + str(int(PSevs['sta'].iloc[i]))
        
    elif (not isinstance(PSevs['sta'].iloc[i],str)) and (math.isnan(PSevs['sta'].iloc[i])):
        pass
    
    else:
        PSevs['sta'].iloc[i] = str(int(PSevs['sta'].iloc[i]))

PSevs = PSevs.reset_index(drop=True)

# setting station name to PR + station number

for i in np.arange(len(PSevs)):
    
    if isinstance(PSevs['sta'].iloc[i],str):
      if int(PSevs['sta'].iloc[i]) < 10:
        PSevs['sta'].iloc[i] = 'PR0' + PSevs['sta'].iloc[i] + "I" + "P"
      else:
        PSevs['sta'].iloc[i] = 'PR' + PSevs['sta'].iloc[i] + "I" + "P"
    
    else:
        pass

PSobs['sta'] = PSevs['sta']
PSobs['date(yyyymmdd)'] = PSevs['time']
PSobs['hhmm'] = PSevs['time']
PSobs['ss'] = PSevs['time']

# defining station, instrument, comp, Ponset, phase, firstmotion, date, err, coda_duration, amplitude and period
for i in np.arange(0,len(PSevs)):
    
    if PSobs['sta'].values[i] == str(PSobs['sta'].values[i]):

        PSobs['sta'].values[i] = PSevs['sta'].values[i][0:4]
        
        PSobs['instrument'].values[i] = '?'
        PSobs['comp'].values[i] = '?'
        PSobs['Ponset'].values[i] = '?'
        PSobs['phase'].values[i] = PSevs['pha'].values[i]
        PSobs['firstmotion'].values[i] = '?'
        PSobs['date(yyyymmdd)'].values[i] = '20' + PSevs['time'].values[i][0:6] + PSevs['time'].values[i][4:6]
        PSobs['hhmm'].values[i] = PSevs['time'].values[i][6:10]
        PSobs['ss'].values[i] = PSevs['time'].values[i][10:]
        PSobs['err'].values[i] = 'GAU'
        PSobs['errmag'].values[i] = '?'
        PSobs['coda_duration'].values[i] = -1.00e+00
        PSobs['amplitude'].values[i] =-1.00e+00
        PSobs['period'].values[i] =-1.00e+00  
        
    else:
        pass
    
# defining probability ranking and errmag columns
PSobs['prob'] = PSevs['prob']
PSobs['prob_rank']= np.arange(0,len(PSobs))
prob_rank = []
errmag = []
for i in np.arange(0,len(PSevs)):

    if PSobs['prob'].values[i] >= 0.85:
        prob_rank.append(0)
        errmag.append(5*1/2000)

    elif PSobs['prob'].values[i] >=0.7:
        prob_rank.append(1)
        errmag.append(10*1/2000)
        
    elif PSobs['prob'].values[i] >= 0.6:
        prob_rank.append(2)
        errmag.append(20*1/2000)

    elif PSobs['prob'].values[i] >= 0.5:
        prob_rank.append(3)
        errmag.append(50*1/2000)
        
    elif PSobs['prob'].values[i] < 0.5:
        prob_rank.append(4)
        errmag.append(99999.9)
        
    elif math.isnan(PSobs['prob'].values[i]):
        prob_rank.append(np.NaN)
        errmag.append(np.NaN)
PSobs['prob_rank'] = prob_rank
PSobs['errmag'] = errmag


In [ ]:
for i in np.arange(0,len(PSobs)):
   if not isinstance(PSobs['sta'].iloc[i],str):
      PSobs['date(yyyymmdd)'].iloc[i] = np.nan
      PSobs['hhmm'].iloc[i] = np.nan
      PSobs['ss'].iloc[i] = np.nan
   else:
      pass

print('Saving hpf file.')
PSobs[['sta','instrument','comp','Ponset','phase','firstmotion','date(yyyymmdd)','hhmm','ss','err','errmag','coda_duration','amplitude','period']].to_csv(drive +  '/testNLLOC_OBS.hpf', header=None, index=None, sep=' ')

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Saving hpf file.


## NLLOC

In [ ]:
######################
# NLLOC 
######################

# download NLLoc
!wget http://alomax.free.fr/nlloc/soft7.00/tar/NLL7.00_src.tgz; tar -zxf NLL7.00_src.tgz; cd src; make distrib

# make directories

!mkdir ./NLLoc
!mkdir ./NLLoc/picks
!mkdir ./NLLoc/model
!mkdir ./NLLoc/time
!mkdir ./NLLoc/loc

--2021-07-21 15:50:56--  http://alomax.free.fr/nlloc/soft7.00/tar/NLL7.00_src.tgz
Resolving alomax.free.fr (alomax.free.fr)... 212.27.63.116
Connecting to alomax.free.fr (alomax.free.fr)|212.27.63.116|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407847 (398K) [application/x-gzip]
Saving to: ‘NLL7.00_src.tgz’

NLL7.00_src.tgz     100%[===================>] 398.29K   314KB/s    in 1.3s    

2021-07-21 15:50:59 (314 KB/s) - ‘NLL7.00_src.tgz’ saved [407847/407847]

gcc -c -O3 -Wall -std=gnu99  NLLoc_main.c  
gcc -c -O3 -Wall -std=gnu99  NLLoc1.c  
NLLoc1.c: In function ‘NLLoc’:
NLLoc1.c:277:9: warning: ignoring return value of ‘system’, declared with attribute warn_unused_result [-Wunused-result]
         system(sys_command);
         ^~~~~~~~~~~~~~~~~~~
NLLoc1.c:280:9: warning: ignoring return value of ‘system’, declared with attribute warn_unused_result [-Wunused-result]
         system(sys_command);
         ^~~~~~~~~~~~~~~~~~~
NLLoc1.c:612:17: warning: ignor

In [ ]:
# blanketPScindy_nll.in
! ./src/Vel2Grid ./PNR_run/blanketPScindy_nll.in

Vel2Grid (NonLinLoc v7.00.00 27Oct2017) 
CONTROL:  MessageFlag: 3  RandomNumSeed: 54321
TRANSFORM  SIMPLE LatOrig 53.775770  LongOrig -2.987780  RotCW 0.000000
Vel2Grid files:  Output: ./NLLoc/model/3km0.025.*
Vel2Grid wave type:  P
Vel2Grid wave type:  S
GRID: {x, y, z}
  Num: {121, 121, 121}
  Orig: {0, 0, 0}
  LenSide: {0.025, 0.025, 0.025}
  Type: SLOW_LEN
Creating model grid files: ./NLLoc/model/3km0.025.P.mod.*
Creating model grid files: ./NLLoc/model/3km0.025.S.mod.*


In [ ]:
# Pg2tblanketPScindy_nll.in
! ./src/Grid2Time ./PNR_run/Pg2tblanketPScindy_nll.in

# Sg2tblanketPScindy_nll.in
! ./src/Grid2Time ./PNR_run/Sg2tblanketPScindy_nll.in

Streaming output truncated to the last 5000 lines.
update side z=110: ff fb bb bf z=110 <R#1>updated.
update side z=109: ff fb bb bf z=109 <R#1>updated.
update side z=108: ff fb bb bf z=108 <R#1>updated.
update side z=107: ff fb bb bf z=107 <R#1>updated.
update side z=106: ff fb bb bf z=106 <R#1>updated.
update side z=105: ff fb bb bf z=105 <R#1>updated.
update side z=104: ff fb bb bf z=104 <R#1>updated.
update side z=103: ff fb bb bf z=103 <R#1>updated.
update side z=102: ff fb bb bf z=102 <R#1>updated.
update side z=101: ff fb bb bf z=101 <R#1>updated.
update side z=100: ff fb bb bf z=100 <R#1>updated.
update side z=99: ff fb bb bf z=99 <R#1>updated.
update side z=98: ff fb bb bf z=98 <R#1>updated.
update side z=97: ff fb bb bf z=97 <R#1>updated.
update side z=96: ff fb bb bf z=96 <R#1>updated.
update side z=95: ff fb bb bf z=95 <R#1>updated.
update side z=94: ff fb bb bf z=94 <R#1>updated.
update side z=93: ff fb bb bf z=93 <R#1>updated.
update side z=92: ff fb bb bf z=92 <R#1>updat

In [ ]:
# NLLOC from code before
os.system('sed "s|LOCFILES ./NLLOC_OBS/PS_181015_NLLOC_OBS.hpf NLLOC_OBS ./NLLOC/time/3km0.025 ./NLLoc/loc/181015/ALLPS|LOCFILES ./PNR_run/testNLLOC_OBS.hpf NLLOC_OBS ./NLLoc/time/3km0.025 ./NLLoc/loc/ALLPS|g" "./PNR_run/blanketPScindy_nll.in" > "./PNR_run/blanketPScindy_best.in"')

start = datetime.datetime.now()
!./src/NLLoc ./PNR_run/blanketPScindy_best.in
end = datetime.datetime.now()
timing = end - start
print(timing)


Streaming output truncated to the last 5000 lines.
Opening Grid File: ./NLLoc/time/3km0.025.P.PR09.time.buf
GridMemManager: Grid exists in mem (19/48): ./NLLoc/time/3km0.025.P.PR09.time
Checking Arrival 19:  PR07 (PR07)  ? p ? 0
Opening Grid File: ./NLLoc/time/3km0.025.p.PR07.time.buf
Opening Grid File: ./NLLoc/time/3km0.025.P.PR07.time.buf
GridMemManager: Grid exists in mem (25/48): ./NLLoc/time/3km0.025.P.PR07.time
Checking Arrival 20:  PR05 (PR05)  ? p ? 0
Opening Grid File: ./NLLoc/time/3km0.025.p.PR05.time.buf
Opening Grid File: ./NLLoc/time/3km0.025.P.PR05.time.buf
GridMemManager: Grid exists in mem (17/48): ./NLLoc/time/3km0.025.P.PR05.time
Checking Arrival 21:  PR03 (PR03)  ? p ? 0
Opening Grid File: ./NLLoc/time/3km0.025.p.PR03.time.buf
Opening Grid File: ./NLLoc/time/3km0.025.P.PR03.time.buf
GridMemManager: Grid exists in mem (10/48): ./NLLoc/time/3km0.025.P.PR03.time
Checking Arrival 22:  PR04 (PR04)  ? p ? 0
Opening Grid File: ./NLLoc/time/3km0.025.p.PR04.time.buf
Opening G

In [ ]:
##############################
# NLLOC output --> catalogue
##############################

# path to
origins = []
lats = []
lons = []
depths = []

with open('./NLLoc/loc/ALLPS.sum.grid0.loc.hyp','r') as fi:
  for ln in fi:
    if ln.startswith('GEOGRAPHIC'):
      origins.append(ln[15:42])
      lats.append(float(ln[47:57]))
      lons.append(float(ln[62:72]))
      depths.append(float(ln[78:]))

# loop through and convert origin times to UTCDateTime
for i in range(len(origins)):
  origins[i] = UTCDateTime(origins[i][:4] + origins[i][5:7] + origins[i][8:10] + 'T' + origins[i][12:14] + origins[i][15:17] + "%.4f" % float(origins[i][18:27].zfill(7)))

# create a dataframe of event locations
dfs = pd.DataFrame(list(zip(origins,lats,lons,depths)), columns = ['time', 'lat', 'lon', 'depth'])
# Define the wgs84 and osgb36 projection
wgs84 = pyproj.CRS("EPSG:4326")
osgb = pyproj.CRS("EPSG:27700")

lat = dfs['lat']
lon = dfs['lon']
xx, yy = pyproj.transform(wgs84,osgb,lat,lon)
dfs['xx'] = xx
dfs['yy'] = yy

# # uncomment to save
dfs.to_csv(drive+ '/Catalog/UGPD_PNR_catalogue.csv')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


In [ ]:
dfs

,time,lat,lon,depth,xx,yy
0,2018-12-11T09:15:05.208900Z,53.788426,-2.963241,2.380469,336636.843141,432887.050384
1,2018-12-11T09:15:10.594100Z,53.788273,-2.962462,1.967969,336687.933355,432869.332244
2,2018-12-11T09:15:58.875200Z,53.787659,-2.965578,2.397656,336481.719438,432803.809456
3,2018-12-11T09:16:02.477200Z,53.787928,-2.965643,2.393359,336477.844227,432833.795318
4,2018-12-11T09:17:57.064200Z,53.788119,-2.968175,2.346094,336311.322396,432857.317156
...,...,...,...,...,...,...
1155,2018-12-11T09:59:54.351600Z,53.788810,-2.965188,2.285938,336509.154466,432931.514625
1156,2018-12-11T09:59:56.802300Z,53.788158,-2.967720,2.281641,336341.357434,432861.247364
1157,2018-12-11T09:59:57.532200Z,53.788388,-2.965123,2.376172,336512.798345,432884.506758
1158,2018-12-11T09:59:58.375700Z,53.788503,-2.967785,2.285938,336337.598456,432899.688736


____